## Module 4 Project - H1N1 Flu Shot ##
This project will be over the driven data competition over the H1N1 Flu shot dataset.

I will explore the data and create a model to predict what behavioral features affect the most if a person gets a vaccination for the flu or not.

This Notebook is meant to combine the raw datasets and clean them up for modeling.

Importing all Necessary Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

First loading all 4 of the datasets given to us by the competition

In [2]:
df = pd.read_csv('CSVs/training_set_features.csv') #First half of the raw data

In [3]:
df2 = pd.read_csv('CSVs/training_set_labels.csv') #First half of the results data

Now to combine the two datasets to get the full rows of the data of 53415
And then combine the two into one final dataset we will start the cleaning of the data from

In [4]:
df_combined = df.merge(df2,on='respondent_id') 
#Combining everything together into one dataset based off of the Respondent ID

Doing a quick check of what the dataset looks like so far

In [5]:
pd.options.display.max_columns = None #Turning on the option to see all columns in the dataset

In [6]:
df_combined

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,1.0,1.0,5.0,2.0,2.0,65+ Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN,0,0
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea,0,0
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,4.0,2.0,5.0,4.0,2.0,55 - 64 Years,Some College,White,Female,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN,0,1
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,2.0,1.0,2.0,18 - 34 Years,Some College,Hispanic,Female,"<= $75,000, Above Poverty",Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg,0,0


Before I look into the data I am going to replace the index with the respondent_id column.

In [7]:
df_combined.set_index('respondent_id',inplace=True)

In [8]:
df_combined.info()

#Looking to see what type of data each column is. 
#Anything listed Object will need to be changed prior to modeling

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 37 columns):
h1n1_concern                   26615 non-null float64
h1n1_knowledge                 26591 non-null float64
behavioral_antiviral_meds      26636 non-null float64
behavioral_avoidance           26499 non-null float64
behavioral_face_mask           26688 non-null float64
behavioral_wash_hands          26665 non-null float64
behavioral_large_gatherings    26620 non-null float64
behavioral_outside_home        26625 non-null float64
behavioral_touch_face          26579 non-null float64
doctor_recc_h1n1               24547 non-null float64
doctor_recc_seasonal           24547 non-null float64
chronic_med_condition          25736 non-null float64
child_under_6_months           25887 non-null float64
health_worker                  25903 non-null float64
health_insurance               14433 non-null float64
opinion_h1n1_vacc_effective    26316 non-null float64
opinion_h1n1_risk          

In [9]:
(df_combined.isna().sum()/len(df_combined))*100
#Getting the Precentage of null values

h1n1_concern                    0.344479
h1n1_knowledge                  0.434343
behavioral_antiviral_meds       0.265848
behavioral_avoidance            0.778822
behavioral_face_mask            0.071142
behavioral_wash_hands           0.157262
behavioral_large_gatherings     0.325757
behavioral_outside_home         0.307036
behavioral_touch_face           0.479275
doctor_recc_h1n1                8.087767
doctor_recc_seasonal            8.087767
chronic_med_condition           3.635751
child_under_6_months            3.070356
health_worker                   3.010447
health_insurance               45.957989
opinion_h1n1_vacc_effective     1.464036
opinion_h1n1_risk               1.452803
opinion_h1n1_sick_from_vacc     1.479013
opinion_seas_vacc_effective     1.729884
opinion_seas_risk               1.924589
opinion_seas_sick_from_vacc     2.010709
age_group                       0.000000
education                       5.268282
race                            0.000000
sex             

## So a quick look at the dataset nulls: ##

* Three of the columns has close to or half of the data missing.
    * The employement_industry and occupation data seem to be more determintal than productive so I am going to remove those.
    * The health_insurance data will be replaced with the mean value of the remaining data.
* The following columns will be removed since it doesn't do too much to help my analysis:
    * income_poverty
    * marital_status
    * rent_or_own
    * employment_status
    * hhs_geo_region
    * census_msa
    * household_adults
    * household_children
    * seasonal_vaccine (I would like to do analysis on this as well but sadly I ran out of time so I am getting rid of it)
* The following columns need to be encoded for usage in the modeling.
    * age_group
    * education
    * race
    * Sex

In [10]:
df_combined.drop(columns=["income_poverty", "marital_status", "rent_or_own", "employment_status", "hhs_geo_region", "census_msa", "household_adults", "household_children", "employment_industry", "employment_occupation", "seasonal_vaccine","doctor_recc_seasonal","health_worker","opinion_seas_vacc_effective","opinion_seas_risk", "opinion_seas_sick_from_vacc"],inplace=True)


In [11]:
le = LabelEncoder() #Encoding the string values into numbers between 0 - 5
df_combined['Age'] = le.fit_transform(df_combined['age_group'])
df_combined['Race'] = le.fit_transform(df_combined['race'])
df_combined['Gender'] = le.fit_transform(df_combined['sex'])

In [12]:
# Last column needing encoding but I need to remove null values first.
# Sadly most of the data is relatively spread out (making getting the mean value somewhat difficult) 
# and the portion of null data is small (5.27%) I am going to get rid of the null rows.

df_combined = df_combined.dropna(subset = ['education'])
df_combined['Education'] = le.fit_transform(df_combined['education'])

In [13]:
# Now to get rid of the duplicate original rows
df_combined.drop(columns=['age_group','race','sex','education'],inplace=True)

In [14]:
#Now to tackle the final biggest null value column: health_insurance
#Due to the importance that insurance would play for this particular problem I am going to drop all null values.
df_combined = df_combined.dropna(subset = ['health_insurance'])

In [15]:
#Since most of the remaining null values were small portions of the data I am replacing it with the median values of their respective columns
df_combined['h1n1_concern'].fillna(df_combined['h1n1_concern'].median(),inplace=True)
df_combined['h1n1_knowledge'].fillna(df_combined['h1n1_knowledge'].median(),inplace=True)
df_combined['behavioral_antiviral_meds'].fillna(df_combined['behavioral_antiviral_meds'].median(),inplace=True)
df_combined['behavioral_avoidance'].fillna(df_combined['behavioral_avoidance'].median(),inplace=True)
df_combined['behavioral_face_mask'].fillna(df_combined['behavioral_face_mask'].median(),inplace=True)
df_combined['behavioral_wash_hands'].fillna(df_combined['behavioral_wash_hands'].median(),inplace=True)
df_combined['behavioral_large_gatherings'].fillna(df_combined['behavioral_large_gatherings'].median(),inplace=True)
df_combined['behavioral_outside_home'].fillna(df_combined['behavioral_outside_home'].median(),inplace=True)
df_combined['behavioral_touch_face'].fillna(df_combined['behavioral_touch_face'].median(),inplace=True)
df_combined['doctor_recc_h1n1'].fillna(df_combined['doctor_recc_h1n1'].median(),inplace=True)
df_combined['chronic_med_condition'].fillna(df_combined['chronic_med_condition'].median(),inplace=True)
df_combined['child_under_6_months'].fillna(df_combined['child_under_6_months'].median(),inplace=True)
df_combined['opinion_h1n1_vacc_effective'].fillna(df_combined['opinion_h1n1_vacc_effective'].median(),inplace=True)
df_combined['opinion_h1n1_risk'].fillna(df_combined['opinion_h1n1_risk'].median(),inplace=True)
df_combined['opinion_h1n1_sick_from_vacc'].fillna(df_combined['opinion_h1n1_sick_from_vacc'].median(),inplace=True)

In [16]:
# Final check of the dataset
df_combined

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,chronic_med_condition,child_under_6_months,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,h1n1_vaccine,Age,Race,Gender,Education
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,0,3,3,0,1
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,0,1,3,1,0
7,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,5.0,2.0,1.0,1,2,3,0,3
9,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,4.0,2.0,2.0,0,3,3,1,0
10,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0,1.0,2.0,1,2,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26699,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,1.0,5.0,0,2,3,0,3
26701,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,4.0,2.0,4.0,0,0,3,0,2
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,0,0,3,1,2


In [17]:
# Check to see if there are any null values left
df_combined.isna().sum()

h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
chronic_med_condition          0
child_under_6_months           0
health_insurance               0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
h1n1_vaccine                   0
Age                            0
Race                           0
Gender                         0
Education                      0
dtype: int64

Now to create a new dataset with the desired columns.

In [18]:
df_combined.to_csv(r'CSVs/Finalized_Dataset.csv')